# Creating epochs and generating evoked responses (ERP/ERF)

In [ ]:
import pathlib
import matplotlib

import mne
import mne_bids

matplotlib.use('Qt5Agg')

In [ ]:
bids_root = pathlib.Path('out_data/sample_BIDS')

bids_path = mne_bids.BIDSPath(subject='01',
                              session='01',
                              task='audiovisual',
                              run='01',
                              datatype='meg',
                              root=bids_root)

raw = mne_bids.read_raw_bids(bids_path)
raw.load_data()
raw.filter(l_freq=0.1, h_freq=40)
events, event_id = mne.events_from_annotations(raw)

In [ ]:
event_id

In [ ]:
tmin = -0.3
tmax = 0.5
baseline = (None, 0)

epochs = mne.Epochs(raw,
                    events=events,
                    event_id=event_id,
                    tmin=tmin,
                    tmax=tmax,
                    baseline=baseline,
                    preload=True)
epochs

In [ ]:
epochs.plot()

<div class="alert alert-success">
    <b>EXERCISE</b>:
     <ul>
         <li>Create epochs starting 250 ms before the stimulus onset and ending 800 ms after stimulus onset, and apply baseline correctin with a baseline period ranging from -200 to 0 ms.</li>
    </ul>
</div>

## Selecting epochs based on experimental conditions

In [ ]:
epochs['Auditory/Right']

In [ ]:
epochs['Auditory']

In [ ]:
epochs['Left']

In [ ]:
epochs['Visual'].plot_image()

<div class="alert alert-success">
    <b>EXERCISE</b>:
     <ul>
         <li>Extract all epochs with a "Right" condition.</li>
    </ul>
</div>

## Saving epochs

In [ ]:
epochs.save(pathlib.Path('out_data') / 'epochs_epo.fif', 
            overwrite=True)

## Creating evoked data

In [ ]:
evoked_auditory = epochs['Auditory'].average()
evoked_visual = epochs['Visual'].average()

In [ ]:
evoked_auditory.plot(spatial_colors=True)

In [ ]:
evoked_auditory.plot_topomap(ch_type='mag')

In [ ]:
evoked_auditory.plot_joint(picks='mag')

In [ ]:
mne.viz.plot_compare_evokeds([evoked_auditory, evoked_visual], picks='mag')

<div class="alert alert-success">
    <b>EXERCISE</b>:
     <ul>
         <li>Plot a GFP comparison for the "Visual/Left" and "Visual/Right" conditions of the EEG data.</li>
    </ul>
</div>

## Saving evoked data

In [ ]:
mne.write_evokeds(fname=pathlib.Path('out_data') / 'evokeds_ave.fif',
                  evoked=[evoked_auditory, evoked_visual])

## Reading evoked data

In [ ]:
evokeds = mne.read_evokeds(fname=pathlib.Path('out_data') / 'evokeds_ave.fif')
evokeds

In [ ]:
evokeds[0]

In [ ]:
evoked = mne.read_evokeds(fname=pathlib.Path('out_data') / 'evokeds_ave.fif',
                          condition='0.50 * Visual/Left + 0.50 * Visual/Right')
evoked